# Parsing and Interpreting Data From [FLUORITE](http://www.cs.cmu.edu/~fluorite/)

FLUORITE (Full of Low-level User Operations Recorded In The Editor) is a plugin for the Eclipse IDE that records low-level file operations from the IDE. The log records a wealth of information, including data which is not fully leveraged by this project. The `fluorite` Python module's sole purpose is to approximately reconstruct the files as they would have appeared at any time while the FLUORITE plugin was recording.

## Contents
* [Parsing a FLUORITE Log File](#Parsing-a-FLUORITE-Log-File)
* [Reconstructing Project Files From a Timestamp](#Reconstructing-Project-Files-From-a-Timestamp)
* [Saving a Timeline](#Saving-a-Timeline)

In [1]:
from fluorite import ProjectHistory

## Parsing a FLUORITE Log File

Logs from FLUORITE are all in XML format. Information such as edit times and locations are contained in attributes, and the types of edits or interactions with the editor are denoted with tag names.

Constructing a `ProjectHistory` object parses the log file and creates a list of document changes for each file.

In [2]:
fluorite_log = "sample-data/log-files/Log-sample-2.xml"
project_history = ProjectHistory(fluorite_log)

## Reconstructing Project Files From a Timestamp

On each call to `get_snapshot`, the ProjectHistory object applies all the changes up to the given timestamp. This is less than optimal, but it works well in the normal case, where a relatively low number of edits are made.

In [3]:
# Get the time at which the IDE started up
launch_time = project_history.launch_time

# Pick a time in milliseconds at which to view the file.
# Note that this will return a tuple that also contains the current state
#    of any function/entity indices you may have used.
content_1 = project_history.get_snapshot("Foo.java", launch_time + 85000)[0]
print(content_1)

print("\n---------------------------------\n")

# Pick another time. Note that the IDE was closed after this time, but
#      no exception is raised.
content_2 = project_history.get_snapshot("Foo.java", launch_time + 10**7)[0]
print(content_2)

public class Foo{
	
}

---------------------------------

public class Foo{
	private int foods;
	private String attitude;
	
	public Foo(int starting_foods) {
		this.foods = starting_foods;
		this.attitude = foods < 5? "Grumpy": "Grudgingly Satisfied";
	}
}


## Saving a Timeline

You may wish to save several copies of the project as they existed at different times. This can be done in various ways.

#### Method 1: Save As Many Snapshots As Possible

In [4]:
# This will save a set of files that cover EVERY change that was made 
#     while FLUORITE was recording.
project_history.save_timeline(
    "sample-timeline-finest", 
    granularity="finest",
    first_time=launch_time,
    last_time=launch_time+10**7 # Required, but will not change results
                                #   if greater than the duration of the session.
)

# A list of periods will be returned. These are the periods during which
#    the corresponding segment of the timeline was valid.

[[1565111420525, 1565111487334],
 [1565111489858, 1565111490830],
 [1565111491543, 1565111492344],
 [1565111492861, 1565111493138],
 [1565111493138, 1565111493285],
 [1565111495319, 1565111496202],
 [1565111497636, 1565111497918],
 [1565111498167, 1565111498476],
 [1565111498476, 1565111498642],
 [1565111498714, 1565111498964],
 [1565111499120, 1565111499227],
 [1565111502898, 1565111503467],
 [1565111503467, 1565111503663],
 [1565111503663, 1565111504986],
 [1565111505180, 1565111507696],
 [1565111509576, 1565111510074],
 [1565111510074, 1565111511104],
 [1565111511584, 1565111511998],
 [1565111512338, 1565111512573],
 [1565111514655, 1565111515226],
 [1565111515954, 1565111517015],
 [1565111517015, 1565111517486],
 [1565111517486, 1565111518413],
 [1565111519246, 1565111519619],
 [1565111519619, 1565111519735],
 [1565111524047, 1565111524995],
 [1565111524995, 1565111525690],
 [1565111525690, 1565111529201],
 [1565111531879, 1565111534377],
 [1565111534377, 1565111534987],
 [15651115

#### Method 2: Save Snapshots at a Given Interval

In [5]:
# Alternatively, you can control the interval at which snapshots are saved.
# You need to specify the starting and ending times for this one,
#    and they need to be correct.
project_history.save_timeline(
    "sample-timeline-periodic",   # Output directory
    granularity=10000,            # Time step in ms
    first_time=launch_time,
    last_time=launch_time+180000  # Must be correct, otherwise too many
                                  #    or too few snapshots will be saved.
)

# Periods are not returned, but a smaller timeline is saved.